# Hashing w/ Hamming Distance

In [13]:
import cv2 
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import time
import sys

# Import matplotlib modules
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

import binascii  # For converting hashes to hex
from tkinter import filedialog

# Initialize variables for video source
use_video_file = True  # Set to True to use a video file instead of the webcam
video_file_name = 'OurPlanet.mp4'  # Replace with your video file name

# Construct the path to the video file relative to the script directory
video_file_path = os.path.join(os.getcwd(), video_file_name)

# Initialize video capture based on the selected source
if use_video_file:
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_file_name}.")
        exit()
else:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        exit()

ret, frame = cap.read()
if not ret:
    print("Error: Could not read from the video source.")
    exit()

# Get the frame dimensions
frame_height, frame_width = frame.shape[:2]

# Knowledge base to store hashes of previously seen frames
knowledge_base_hashes = set()

# Tkinter setup
window = tk.Tk()
window.title("Frame Similarity Detection")

# Optionally set a minimum size for the window
window.minsize(800, 600)

# Tkinter variables
seen_before_accuracy_var = tk.StringVar(value="Similarity: 0.00%")
resolution_var = tk.StringVar()
live_framerate_var = tk.StringVar(value="Live Framerate: 0.00 FPS")
knowledge_base_size_var = tk.StringVar(value="Knowledge Base Size: 0 bytes")
similarity_threshold = tk.DoubleVar(value=50.0)

# Set initial value for resolution
resolution_var.set(f"Resolution: {frame_width}x{frame_height}")

# Create frames for grouping widgets
video_frame = tk.Frame(window)
video_frame.grid(row=0, column=0, columnspan=3, padx=5, pady=5)

stats_frame = tk.Frame(window)
stats_frame.grid(row=1, column=0, columnspan=3, padx=5, pady=2)

info_frame = tk.Frame(window)
info_frame.grid(row=2, column=0, columnspan=3, padx=5, pady=2)

threshold_frame = tk.Frame(window)
threshold_frame.grid(row=3, column=0, columnspan=3, padx=5, pady=2)

plot_frame = tk.Frame(window)
plot_frame.grid(row=4, column=0, columnspan=3, padx=5, pady=2)

button_frame = tk.Frame(window)
button_frame.grid(row=5, column=0, columnspan=3, padx=5, pady=5)

# Video display
original_label = tk.Label(video_frame)
original_label.pack()

# Stats labels (Similarity and Knowledge Base Size)
accuracy_label = tk.Label(stats_frame, textvariable=seen_before_accuracy_var, font=("Helvetica", 12))
accuracy_label.pack(side=tk.LEFT, padx=5)

knowledge_base_size_label = tk.Label(stats_frame, textvariable=knowledge_base_size_var, font=("Helvetica", 12))
knowledge_base_size_label.pack(side=tk.LEFT, padx=5)

# Info labels (Resolution and Live Framerate)
resolution_label = tk.Label(info_frame, textvariable=resolution_var, font=("Helvetica", 12))
resolution_label.pack(side=tk.LEFT, padx=5)

live_framerate_label = tk.Label(info_frame, textvariable=live_framerate_var, font=("Helvetica", 12))
live_framerate_label.pack(side=tk.LEFT, padx=5)

# Similarity threshold slider
threshold_label = tk.Label(threshold_frame, text="Similarity Threshold (%)", font=("Helvetica", 12))
threshold_label.pack(side=tk.TOP, pady=2)

threshold_slider = tk.Scale(threshold_frame, from_=0, to=100, orient=tk.HORIZONTAL,
                            variable=similarity_threshold, length=400)
threshold_slider.pack(side=tk.TOP, pady=2)

# Plot setup
fig = Figure(figsize=(6, 3), dpi=100)  # Adjusted figsize
ax = fig.add_subplot(111)
ax.set_title("Similarity Over Time")
ax.set_xlabel("Frame")
ax.set_ylabel("Similarity (%)")

canvas = FigureCanvasTkAgg(fig, master=plot_frame)
canvas.draw()
canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Ensure the plot_frame expands to fill available space
plot_frame.grid_rowconfigure(0, weight=1)
plot_frame.grid_columnconfigure(0, weight=1)

# Buttons
export_button = tk.Button(button_frame, text="Export Knowledge Base", command=lambda: export_knowledge_base(knowledge_base_hashes))
export_button.pack(side=tk.LEFT, padx=5)

import_button = tk.Button(button_frame, text="Import Knowledge Base", command=lambda: import_knowledge_base(knowledge_base_hashes))
import_button.pack(side=tk.LEFT, padx=5)

close_button = tk.Button(button_frame, text="Close", command=window.destroy)
close_button.pack(side=tk.LEFT, padx=5)

# Initialize time for framerate calculation
last_frame_time = time.time()

# Initialize similarity history
similarity_history = []

# Function definitions

def calculate_ahash(frame, hash_size=8):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (hash_size, hash_size))
    avg_pixel_value = np.mean(resized_frame)
    ahash = ''.join(['1' if pixel > avg_pixel_value else '0' for pixel in resized_frame.flatten()])
    return ahash

def hamming_distance(hash1, hash2):
    return sum(el1 != el2 for el1, el2 in zip(hash1, hash2))

def get_min_hamming_distance(frame_hash, knowledge_base_hashes):
    min_distance = 64  # Max possible distance for 8x8 hash
    for stored_hash in knowledge_base_hashes:
        distance = hamming_distance(frame_hash, stored_hash)
        if distance < min_distance:
            min_distance = distance
    return min_distance

def get_knowledge_base_size(knowledge_base_hashes):
    total_size = sys.getsizeof(knowledge_base_hashes)
    for h in knowledge_base_hashes:
        total_size += sys.getsizeof(h)
    return total_size

def export_knowledge_base(knowledge_base_hashes):
    if not knowledge_base_hashes:
        print("Knowledge base is empty. Nothing to export.")
        return
    # Ask the user for a file name and location to save the knowledge base
    export_filename = filedialog.asksaveasfilename(defaultextension=".txt",
                                                   filetypes=[("Text files", "*.txt"), ("All files", "*.*")],
                                                   title="Save Knowledge Base As")
    if not export_filename:
        print("Export cancelled.")
        return
    try:
        with open(export_filename, 'w') as f:
            for h in knowledge_base_hashes:
                # Convert the binary hash string to hexadecimal representation
                h_hex = hex(int(h, 2))[2:]  # Remove '0x' prefix
                f.write(h_hex + '\n')
        print(f"Knowledge base exported to {export_filename}")
    except Exception as e:
        print(f"An error occurred while exporting: {e}")

def import_knowledge_base(knowledge_base_hashes):
    # Ask the user for the knowledge base file to load
    import_filename = filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("All files", "*.*")],
                                                 title="Open Knowledge Base")
    if not import_filename:
        print("Import cancelled.")
        return
    try:
        with open(import_filename, 'r') as f:
            for line in f:
                h_hex = line.strip()
                if not h_hex:
                    continue
                # Convert hexadecimal representation back to binary hash string
                h_bin = bin(int(h_hex, 16))[2:].zfill(64)  # Ensure it's 64 bits
                knowledge_base_hashes.add(h_bin)
        print(f"Knowledge base imported from {import_filename}")
        # Update the knowledge base size variable
        kb_size_bytes = get_knowledge_base_size(knowledge_base_hashes)
        if kb_size_bytes < 1024:
            knowledge_base_size_var.set(f"Knowledge Base Size: {kb_size_bytes} bytes")
        else:
            kb_size_kb = kb_size_bytes / 1024
            knowledge_base_size_var.set(f"Knowledge Base Size: {kb_size_kb:.2f} KB")
    except Exception as e:
        print(f"An error occurred while importing: {e}")

def update():
    global last_frame_time
    if not cap.isOpened():
        return

    try:
        ret, frame = cap.read()

        if not ret:
            if use_video_file:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret, frame = cap.read()
                if not ret:
                    print("Error: Could not read frame from video file.")
                    window.after(10, update)
                    return
            else:
                print("Error: Could not read frame from webcam.")
                window.after(10, update)
                return

        if not use_video_file:
            frame = cv2.flip(frame, 1)

        # Calculate aHash for the current frame
        frame_hash = calculate_ahash(frame)

        # Calculate similarity percentage
        if not knowledge_base_hashes:
            similarity_percentage = 0.0
            knowledge_base_hashes.add(frame_hash)
            # print("First frame stored in knowledge base.")
        else:
            min_distance = get_min_hamming_distance(frame_hash, knowledge_base_hashes)
            similarity_percentage = (1 - (min_distance / 64)) * 100

            # Get the current threshold from the slider
            threshold = similarity_threshold.get()

            # If similarity is less than threshold, add to knowledge base
            if similarity_percentage < threshold:
                knowledge_base_hashes.add(frame_hash)
                # print("New frame added to knowledge base.")

        # Update Tkinter variables
        seen_before_accuracy_var.set(f"Similarity: {similarity_percentage:.2f}%")

        kb_size_bytes = get_knowledge_base_size(knowledge_base_hashes)
        if kb_size_bytes < 1024:
            knowledge_base_size_var.set(f"Knowledge Base Size: {kb_size_bytes} bytes")
        else:
            kb_size_kb = kb_size_bytes / 1024
            knowledge_base_size_var.set(f"Knowledge Base Size: {kb_size_kb:.2f} KB")

        # Calculate live framerate
        current_time = time.time()
        delta_time = current_time - last_frame_time
        if delta_time > 0:
            live_framerate = 1.0 / delta_time
            live_framerate_var.set(f"Live Framerate: {live_framerate:.2f} FPS")
        else:
            live_framerate_var.set(f"Live Framerate: Calculating...")
        last_frame_time = current_time

        # Resize frame for display purposes
        scale = 0.5
        display_width = int(frame_width * scale)
        display_height = int(frame_height * scale)

        frame_resized = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (display_width, display_height))
        original_image_pil = Image.fromarray(frame_resized)
        original_image_tk = ImageTk.PhotoImage(image=original_image_pil)

        # Update the image in the GUI
        original_label.config(image=original_image_tk)
        original_label.image = original_image_tk

        # Update similarity history and plot
        similarity_history.append(similarity_percentage)
        # Removed the limit on similarity_history
        # if len(similarity_history) > 100:
        #     similarity_history.pop(0)  # Keep the last 100 data points

        ax.clear()
        ax.set_title("Similarity Over Time")
        ax.set_xlabel("Frame")
        ax.set_ylabel("Similarity (%)")
        ax.plot(similarity_history, color='blue')
        # Removed the fixed y-axis limits to allow automatic scaling
        # ax.set_ylim(0, 100)
        canvas.draw()

        window.after(10, update)

    except Exception as e:
        print(f"An error occurred: {e}")
        window.after(10, update)

# Start the update loop
update()

# Start the Tkinter event loop
window.mainloop()


First frame stored in knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added to knowledge base.
New frame added t